## 处理商家的信息

In [49]:
import pandas as pd
import numpy as np
os_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\valid\os_train_shop.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\oc_test.csv')

In [50]:
col='Merchant_id'
a=set(os_train[col].values)
b=set(test[col].values)
print(len(a))
print(len(b))
print(len(a&b))

3697
1559
1230


In [51]:
# for id in (a-b):
#     os_train.drop(list(os_train[os_train['Merchant_id']==id].index),axis=0,inplace=True)
os_train=os_train.sort_values(by=['User_id']).copy()
os_train.index=np.arange(0,len(os_train),1)
test.index=np.arange(0,len(test),1)

In [52]:
test['shop_id']=0
for indtest in np.arange(0,len(test),1):
    indtrain=os_train[os_train['User_id']==test.loc[indtest,'User_id']].index
    testdis=test.loc[indtest,'Distance']
    if len(indtrain)==0:
        continue
    else:
        if testdis not in os_train.loc[indtrain,'Distance'].values:
            test.loc[indtest,'shop_id']=-1
            continue
        else:
            ot=os_train.loc[indtrain]
            test.loc[indtest,'shop_id']=ot[ot['Distance']==testdis]['shop_id'].max()   

In [53]:
i=3000
tet=test[test['Discount_rate']<1]
for disc in set(tet['Discount_rate'].values):
    test.loc[tet[tet['Discount_rate']==disc].index,'shop_id']=i
    i+=1

In [54]:
i=test['shop_id'].max()
for id in set(test[test['shop_id']==-1]['User_id'].values):
    tt=test[(test['shop_id']==-1)&(test['User_id']==id)]
    test.loc[tt.index,'shop_id']=i
    i=i+1

In [55]:
osss_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\oc_train.csv')
osss_test=pd.read_csv(r'D:\Data\TCForNewComer\deal\Same_userid\oc_test.csv')

In [56]:
indx=osss_train[osss_train['Discount_rate']<1].index
discount=osss_train.loc[indx]
osss_train=osss_train.drop(indx,axis=0)
discount.index=np.arange(0,len(discount),1)
osss_train.index=np.arange(0,len(os_train),1)

In [57]:
discount['shop_id']=0
i=3000
for disc in set(discount['Discount_rate'].values):
    discount.loc[discount[discount['Discount_rate']==disc].index,'shop_id']=i
    i+=1
os_train=pd.concat([os_train,discount],axis=0).copy()

In [58]:
os_train.index=np.arange(0,len(os_train),1)
test.index=np.arange(0,len(test),1)

In [90]:
os_train.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\shop\or_train.csv',index=False)
test.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\shop\or_test.csv',index=False)

In [91]:
os_train['num_pc']=0          #店铺发放的优惠券张数       6月份
os_train['num_M100']=0        #店铺发放大于100的优惠券张数
os_train['num_L100']=0          #店铺发放小于100的优惠券张数
os_train['num_M100use']=0         #店铺大于100优惠券被使用的张数
os_train['num_L100use']=0        #店铺小于100优惠券被使用的张数
os_train['num_use']=0            #店铺优惠券总的被使用的张数
os_train['use_pro']=0            #使用占得比率
os_train['cus_shop_num']=0     #同一个用户在同一个店铺领取的张数
os_train['cus_shop_use']=0    #同一个用户在同一个店铺使用的张数
for id in set(os_train['shop_id'].values):
    ot=os_train[os_train['shop_id']==id]
    os_train.loc[ot.index,'num_pc']=len(ot)
    if len(ot[ot['Discount_rate']>=100])!=0:
        os_train.loc[ot[ot['Discount_rate']>=100].index,'num_M100']=len(ot[ot['Discount_rate']>=100])
    if len(ot[ot['Discount_rate']<100])!=0:
        os_train.loc[ot[ot['Discount_rate']<100].index,'num_L100']=len(ot[ot['Discount_rate']<100])
    if len(ot[(ot['Discount_rate']>=100)&(ot['Date']==1)])!=0:
        os_train.loc[ot[(ot['Discount_rate']>=100)&(ot['Date']==1)].index,'num_M100use']=len(ot[(ot['Discount_rate']>=100)&(ot['Date']==1)])
    if len(ot[(ot['Discount_rate']<100)&(ot['Date']==1)])!=0:
        os_train.loc[ot[(ot['Discount_rate']<100)&(ot['Date']==1)].index,'num_L100use']=len(ot[(ot['Discount_rate']<100)&(ot['Date']==1)])
    if len(ot[ot['Date']==1])!=0:
        os_train.loc[ot[ot['Date']==1].index,'num_use']=len(ot[ot['Date']==1])
    os_train.loc[ot.index,'use_pro']=len(ot[ot['Date']==1])/len(ot)
    for uid in set(ot['User_id'].values):
        otu=os_train[(os_train['shop_id']==id)&(os_train['User_id']==uid)]
        os_train.loc[otu.index,'cus_shop_num']=len(otu)
        if len(otu[otu['Date']==1])!=0:
            os_train.loc[otu[otu['Date']==1].index,'cus_shop_use']=len(otu[otu['Date']==1])

In [92]:
test['num_pc']=0          #店铺发放的优惠券张数
test['num_M100']=0        #店铺发放大于100的优惠券张数
test['num_L100']=0          #店铺发放小于100的优惠券张数
test['num_M100use']=0         #店铺大于100优惠券被使用的张数
test['num_L100use']=0        #店铺小于100优惠券被使用的张数
test['num_use']=0            #店铺优惠券总的被使用的张数
test['use_pro']=0            #使用占得比率
test['cus_shop_num']=0     #同一个用户在同一个店铺领取的张数
test['cus_shop_use']=0    #同一个用户在同一个店铺使用的张数
for id in set(test['shop_id'].values):
    ot=test[test['shop_id']==id]
    test.loc[ot.index,'num_pc']=len(ot)
    if len(ot[ot['Discount_rate']>=100])!=0:
        test.loc[ot[ot['Discount_rate']>=100].index,'num_M100']=len(ot[ot['Discount_rate']>=100])
    if len(ot[ot['Discount_rate']<100])!=0:
        test.loc[ot[ot['Discount_rate']<100].index,'num_L100']=len(ot[ot['Discount_rate']<100])
    if id in os_train['shop_id'].values:
        ott=os_train[(os_train['shop_id']==id)]
        if len(ott[(ott['Discount_rate']>=100)&(ott['Date']==1)])!=0:
            test.loc[ot.index,'num_M100use']=ott[(ott['Discount_rate']>=100)&(ott['Date']==1)]['num_M100use'].max()
        if len(ott[(ott['Discount_rate']<100)&(ott['Date']==1)])!=0:
            test.loc[ot.index,'num_L100use']=ott[(ott['Discount_rate']<100)&(ott['Date']==1)]['num_M100use'].max()
        test.loc[ot.index,'num_use']=len(ott[ott['Date']==1])
        test.loc[ot.index,'use_pro']=len(ott[ott['Date']==1])/len(ott)
    for uid in set(ot['User_id'].values):
        otu=test[(test['shop_id']==id)&(test['User_id']==uid)]
        otr=os_train[(os_train['shop_id']==id)&(os_train['User_id']==uid)]
        test.loc[otu.index,'cus_shop_num']=len(otu)
        test.loc[otu.index,'cus_shop_use']=len(otr[otr['Date']==1])

In [93]:
os_train['temp']=os_train['Date_received'].apply(lambda x: int(str(x)[0]))
test['temp']=test['Date_received'].apply(lambda x: int(str(x)[0]))
col=['num_pc','num_M100','num_L100','num_M100use','num_L100use','num_use']
for id in set(os_train['shop_id'].values):
    ot=os_train[os_train['shop_id']==id]
    length=len(set(ot['temp'].values))
    for i in col:
        os_train.loc[ot.index,i]=os_train.loc[ot.index,i]/length
for id in set(test['shop_id'].values):
    ot=test[test['shop_id']==id]
    length=len(set(ot['temp'].values))
    for i in col:
        test.loc[ot.index,i]=test.loc[ot.index,i]/length
for j in col:
    os_train[j]=os_train[j].apply(lambda x: int(x))
    test[j]=test[j].apply(lambda x: int(x))
os_train.drop(['temp'],axis=1,inplace=True)
test.drop(['temp'],axis=1,inplace=True)

In [84]:
os_train.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'ord_buy', 'Coupon_num', 'Coupon_nouse_num',
       'Coupon_use_num', 'Coupon_use_prob', 'Coupon_greaterthan100',
       'Coupon_lessthan100', 'Coupon_lep100', 'Coupon_grp100',
       'Avg_saler_Coupon', 'Avg_distance', 'shop_count', 'shop_pro',
       'Coupon_diffcount', 'Coupon_diffpro', 'same_Coupon_count', 'shop_id',
       'num_pc', 'num_M100', 'num_L100', 'num_M100use', 'num_L100use',
       'num_use', 'use_pro', 'cus_shop_num', 'cus_shop_use', 'temp'],
      dtype='object')

In [83]:
set(os_train['temp'].values)

{1, 2, 3, 4, 5, 6}

In [ ]:
ot=os_train[(os_train['shop_id']==13)&(os_train['Date_received']>=600)]
ot[ot['Date']==1]

In [94]:
os_train.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\shop\shop_train.csv',index=False)
test.to_csv(r'D:\Data\TCForNewComer\deal\Same_userid\shop\shop_test.csv',index=False)

In [79]:
len(test[test['shop_id']==13])

55129